## Make predictions and draw overlays

In [2]:
from pathlib import Path
from skimage.io import imread, imsave
from cellpose import models, io, utils, plot
import os


In [16]:
parent_path = Path(r'D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose')
overlay_path =parent_path /  r'overlay'
model_path = parent_path / 'models'

model = models.CellposeModel(gpu=True, model_type=None, pretrained_model= os.path.join(model_path, 'cellpose_for_imagesc'))

if not Path(overlay_path).exists():
    Path(overlay_path).mkdir()

files = Path(parent_path).glob('*.tif')
files = list(files) + list(Path(parent_path).glob('*.tiff'))

for file in files:
    print(file)
    img = imread(file)
    masks = model.eval(img, channels=[2, 3], cellprob_threshold=0.0, flow_threshold = 1.0)

    img_ = 255*(img/img.max())
    img_ = img_.astype('uint8')

    outlines = utils.masks_to_outlines(masks[0])
    overlay = plot.mask_overlay(img_, masks[0])
    imsave(overlay_path / file.name, overlay)
    #sdfkljsdfsdlk    


D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\aRSI_HF_6_S_2_T_0}RGB_B_DAPI_G_f480_sm.tif
D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\RSI_CD_12_S_1_T_0}RGB_B_DAPI_G_f480-1.tiff
D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\RSI_CD_6_A_1_T_0}RGB_B_DAPI_G_f480-1.tiff
D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\RSI_HF_15_S_2_T_0}RGB_B_DAPI_G_f480-1.tiff
D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\RSI_HF_6_B_1_T_0}RGB_B_DAPI_G_f480-1.tiff
D:\images\tnia-python-images\imagesc\2024_09_29_training_cellpose\RSI_HF_6_S_2_T_0}RGB_B_DAPI_G_f480-1.tiff


In [8]:
print(overlay_path)

D:\overlay


In [22]:
import numpy

def mask_overlay(img, masks, colors=None):
    """Overlay masks on image (set image to grayscale).

    Args:
        img (int or float, 2D or 3D array): Image of size [Ly x Lx (x nchan)].
        masks (int, 2D array): Masks where 0=NO masks; 1,2,...=mask labels.
        colors (int, 2D array, optional): Size [nmasks x 3], each entry is a color in 0-255 range.

    Returns:
        RGB (uint8, 3D array): Array of masks overlaid on grayscale image.
    """
    if colors is not None:
        if colors.max() > 1:
            colors = np.float32(colors)
            colors /= 255
        colors = utils.rgb_to_hsv(colors)
    if img.ndim > 2:
        img = img.astype(np.float32).mean(axis=-1)
    else:
        img = img.astype(np.float32)

    HSV = np.zeros((img.shape[0], img.shape[1], 3), np.float32)
    HSV[:, :, 2] = np.clip((img / 255. if img.max() > 1 else img) * 1.5, 0, 1)
    hues = np.linspace(0, 1, masks.max() + 1)[np.random.permutation(masks.max())]
    for n in range(int(masks.max())):
        ipix = (masks == n + 1).nonzero()
        if colors is None:
            HSV[ipix[0], ipix[1], 0] = hues[n]
        else:
            HSV[ipix[0], ipix[1], 0] = colors[n, 0]
        HSV[ipix[0], ipix[1], 1] = 1.0
    RGB = (utils.hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB